In [2]:
pip install psycopg2

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 22.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [47]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=5437,
    database="db_maisondudroit",
    user="pgis",
    password="pgis"
)

cur = conn.cursor()

In [48]:
df = pd.read_excel("H:/SD3/SAE_DUBOIS_SABINE/PARTIE2/Maison_droit_donnee.xlsx")

In [49]:
display(df)

,NUM,DATE_ENT,MODE_ENT,DUREE,SEXE,AGE,VIENT_PR,SIT_FAM,ENFANT,MODELE_FAM,...,Dem.2,Dem.3,Tot.dem,Sol.1,Sol.2,Sol.3,Tot.Sol,ORIGINE,COMMUNE,PARTENAIRE
0,1,NaN,3,2,1,3,1,7,nc,NaN,...,8c,NaN,NaN,1,7b,NaN,NaN,5a,Vannes K,NaN
1,2,NaN,3,1,2,4,1,5b,1,2,...,NaN,NaN,NaN,1,5c,NaN,NaN,5a,Vannes B,NaN
2,3,NaN,3,1,1,4,6,7,2,NaN,...,NaN,NaN,NaN,6f,NaN,NaN,NaN,1a,Vannes K,NaN
3,4,NaN,3,1,2,5,1,1,0,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,1b,Vannes,NaN
4,5,NaN,3,1,2,3,1,5f,1,1,...,1c,NaN,NaN,1,2b,NaN,NaN,5a,Vannes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,1539,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Vannes K,Défenseur des Droit
1539,1540,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Vannes K,Défenseur des Droit
1540,1541,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Vannes M,Défenseur des Droit
1541,1542,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Vannes M,Défenseur des Droit


In [50]:
insert_entretien = """
INSERT INTO ENTRETIEN (
    NUM, DATE_ENT, MODE, DUREE, SEXE, AGE, VIENT_PR,
    SIT_FAM, ENFANT, MODELE_FAM, PROFESSION, RESS,
    ORIGINE, COMMUNE, PARTENAIRE
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

In [51]:
def to_int_or_none(val):
    if pd.isna(val):
        return None
    if isinstance(val, str) and val.lower() in ["nc"]:
        return None
    try:
        return int(val)
    except:
        return None

In [52]:
def to_int_or_none(val):
    if pd.isna(val):
        return None
    try:
        return int(val)
    except:
        return None


def to_str2_or_none(val):
    if pd.isna(val):
        return None
    return str(val).strip()[:2]


def to_str_or_none(val):
    if pd.isna(val):
        return None
    return str(val).strip()


In [53]:
ok = 0

for i, row in df.iterrows():
    try:
        cur.execute(insert_entretien, (
            int(row["NUM"]),

            row["DATE_ENT"] if pd.notna(row["DATE_ENT"]) else None,

            to_int_or_none(row["MODE_ENT"]),
            to_int_or_none(row["DUREE"]),
            to_int_or_none(row["SEXE"]),
            to_int_or_none(row["AGE"]),
            to_int_or_none(row["VIENT_PR"]),

            # varchar(2)
            to_str2_or_none(row["SIT_FAM"]),

            to_int_or_none(row["ENFANT"]),
            to_str2_or_none(row["MODELE_FAM"]),

            to_int_or_none(row["PROFESSION"]),
            to_int_or_none(row["RESS"]),

            # texte
            to_str_or_none(row["ORIGINE"]),
            to_str_or_none(row["COMMUNE"]),
            to_str_or_none(row["PARTENAIRE"])
        ))

        ok += 1

    except Exception as e:
        conn.rollback()
        print("❌ ERREUR à la ligne Excel :", i)
        print(row.to_dict())
        print("🐘 PostgreSQL dit :", e)
        break


if ok > 0:
    conn.commit()
    print(f"✅ {ok} entretiens insérés")


✅ 1543 entretiens insérés


In [54]:
insert_demande = """
INSERT INTO DEMANDE (num, pos, nature)
VALUES (%s, %s, %s)
"""

In [55]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Dem.1"]

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_demande, (num, 1, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR DEMANDE 1")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"DEMANDE 1 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")

DEMANDE 1 : 1543 lignes insérées
Ignorées : 0


In [56]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Dem.2"]  # <-- on prend la colonne Dem.2

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_demande, (num, 2, str(nature)))  # <-- pos = 2
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR DEMANDE 2")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"DEMANDE 2 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")

DEMANDE 2 : 381 lignes insérées
Ignorées : 1162


In [57]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Dem.3"]  # <- ici c'est Dem.3

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_demande, (num, 3, str(nature)))  # <- pos = 3
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR DEMANDE 3")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"DEMANDE 3 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")

DEMANDE 3 : 55 lignes insérées
Ignorées : 1488


In [58]:
insert_solution = """
INSERT INTO SOLUTION (num, pos, nature)
VALUES (%s, %s, %s)
"""

In [59]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Sol.1"]

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_solution, (num, 1, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR SOLUTION 1")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"SOLUTION 1 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")


SOLUTION 1 : 1543 lignes insérées
Ignorées : 0


In [60]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Sol.2"]

    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_solution, (num, 2, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR SOLUTION 2")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"SOLUTION 2 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")


SOLUTION 2 : 584 lignes insérées
Ignorées : 959


In [61]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Sol.3"]

    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_solution, (num, 3, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR SOLUTION 3")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"SOLUTION 3 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")


SOLUTION 3 : 79 lignes insérées
Ignorées : 1464


In [62]:
def export_table_to_excel(table_name, output_file):
    try:
        cur.execute(f"SELECT * FROM {table_name} ORDER BY num")  
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=colnames)
        df.to_excel(output_file, index=False)
        print(f"Export terminé : {output_file} ({len(df)} lignes)")
    except Exception as e:
        print(f"ERREUR lors de l'export de {table_name} :", e)

#Export des tables 
export_table_to_excel("ENTRETIEN", "entretien_export.xlsx")
export_table_to_excel("DEMANDE", "demande_export.xlsx")
export_table_to_excel("SOLUTION", "solution_export.xlsx")

Export terminé : entretien_export.xlsx (1543 lignes)
Export terminé : demande_export.xlsx (1979 lignes)
Export terminé : solution_export.xlsx (2206 lignes)
